## Generation of MUBs for 2 and 3 qubits

In [19]:
from qiskit.opflow import X, Y, Z, I, PauliOp, PauliSumOp
from qsymm.linalg import simult_diag
import numpy as np
from bqskit.compiler import Compiler
from bqskit.compiler import CompilationTask

In [20]:
import os

path_2_302 = os.path.join(os.getcwd(), 'mub_bqskit', '2_302')
path_3_306 = os.path.join(os.getcwd(), 'mub_bqskit', '3_306')

The method in this section of the notebook is based on the paper "Mutually unbiased binary observable sets on N qubits"
By Lawrence, Brukner and Zeilinger. (2022)

The paper provides a table. In that table, each row contains several Pauli strings.
The basis that this row represents is the eigenvector basis that diagonalizes **the entire row**.

Thus, this function applies simultaneous diagonalization to all matrices (Pauli strings) from each row, to get the MUB.
After that, BQSKit is used to synthesize a circuit that generates those MUB states.

In [3]:
def mats_to_mub_circ(mats_row: np.ndarray, nqubits: int):
    mub_uni = np.hstack(simult_diag(mats_row))
    task = CompilationTask.synthesize(mub_uni)
    with Compiler() as compiler:
        synth_qc = compiler.compile(task)
    return mub_uni, synth_qc.to('qasm')

In [ ]:
tbl_2_302 = [ [Z^I, I^Z, Z^Z],
        [X^I, I^Y, X^Y],
        [Y^I, I^X, Y^X],
        [Y^Y, Z^X, Z^X],
        [X^X, Y^Z, Z^Y]]

tbl_3_306 = [   # The full columns were not added, as to save space, since 3 matrices should define the basis completely.
    [X^I^I, I^Y^I, I^I^Z, X^Y^Z, X^Y^I, X^I^Z, I^Y^Z],  # First 3 rows are product state bases
    [Y^I^I, I^Z^I, I^I^X],
    [Z^I^I, I^X^I, I^I^Y],
    [Y^Z^Z, Z^Y^Z, Z^Z^Y],  # Last 6 rows are GHZ-like bases
    [Z^X^X, X^Z^X, X^X^Z],
    [X^Y^Y, Y^X^Y, Y^Y^X],
    [Z^X^Z, Y^X^X, Y^Y^Z],
    [X^Y^X, Z^Y^Y, Z^Z^X],
    [Y^Z^Y, X^Z^Z, X^X^Y]
]



qasm_2_302 = {}
qasm_3_306 = {}

print('----------RESULTS FOR 2-QUBIT MUBS (3,0,2)--------------')
for i, row in enumerate(tbl_2_302):
    res = mats_to_mub_circ(list(map(lambda p: p.to_matrix(), row)), 2)
    print(f'result for row {i+1}:')
    print(res[0])
    print(res[1])
    with open(os.path.join(path_2_302, str(i+1)+'.txt'), 'w') as f:
        f.write(res[1])
    qasm_2_302[i+1] = res[1]
    print('\n')


print('----------RESULTS FOR 3-QUBIT MUBS (3,0,6)--------------')
for i, row in enumerate(tbl_3_306):
    res = mats_to_mub_circ(list(map(lambda p: p.to_matrix(), row)), 3)
    print(f'result for row {i+1}:')
    print(res[0])
    print(res[1])
    with open(os.path.join(path_3_306, str(i+1) + '.txt'), 'w') as f:
        f.write(res[1])
    qasm_3_306[i+1] = res[1]
    print('\n')

## Barren Plateau Problems

In [21]:
import qiskit as qk
from qiskit import Aer, QuantumCircuit

from qiskit.circuit import Parameter, ClassicalRegister
from qiskit.circuit.library import EfficientSU2
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.algorithms.minimum_eigen_solvers.vqe import VQEResult
from qiskit.algorithms.optimizers import COBYLA
from typing import Tuple, List, Dict, Union
from scipy.optimize import minimize, OptimizeResult
import numpy as np
from random import random
import json

### Basic Barren Plateau Circuit for Variational Quantum Compilation

The ansatz circuit is taken from "Cost Function Dependent Barren Plateaus in Shallow Parametrized Quantum Circuits" by Cerezo et al., 2021, Figure 4.

The attempted task is "trivial" Variational Quantum Compilation.
Vartational Quantum Compilation gets some unitary $U$, and an ansatz $V(\theta)$, and attempts to find a value for $\theta$ such that $V(\theta)| 0 \rangle = U | 0 \rangle$.

In this case, we choose $U=I$.
Because we pick a *random* initial guess for $\theta$, we will experience the barren plateaus that occur when the $\theta$ values are away from the target.

Note, however, an important observation:
In order to actually use the "value" of the different MUB starting points, the original value of the parameters needs to be constant (although random) for all experiments.

I took this specific problem from "Effect of barren plateaus on gradient-free optimization" by Arrasmith et al. (2021).

In [22]:
def gen_vqc_ansatz(n_qubits: int, n_layers: int) -> QuantumCircuit:
    qc = qk.QuantumCircuit(n_qubits)

    idx = 0

    for i in range(n_qubits):
        theta = Parameter(f'theta_{idx}')
        idx += 1
        qc.ry(theta, i)
        

    for layer in range(n_layers):
        for i in range(0, n_qubits-1, 2):
            qc.cz(i, i+1)
        
        for i in range(n_qubits-1):
            theta1 = Parameter(f'theta_{idx}')
            idx += 1
            qc.ry(theta1, i)

        for i in range(1, n_qubits-1, 2):
                qc.cz(i, i+1)
            
        for i in range(1, n_qubits):
            theta2 = Parameter(f'theta_{idx}')
            idx += 1
            qc.ry(theta2, i)

    qc.measure_all()

    return qc


### Experimenting without MUBs

#### Experiment Functions

In [5]:
### Defining experiment values
n_qubits = 3
SHOTS = 8192
MAX_ITER = 10000
backend = Aer.get_backend('qasm_simulator')
qi = QuantumInstance(backend, shots=SHOTS)

no_mub_res = {}

# Returns the number of function evaluations it took for the method to converge.
def run_vqc_exp(qc: QuantumCircuit, n_qubits: int, n_layers: int, theta0: List[float], tol: float = 0.2) -> OptimizeResult:
    def get_val_from_theta(theta: List[float]) -> float:
        concrete_qc = qc.bind_parameters(theta)
        results = qi.execute(concrete_qc)
        return 1 - (results.get_counts().int_raw.get(0, 0) / SHOTS)


    res = minimize(get_val_from_theta,
        theta0,
        method='COBYLA',
        options={'disp': True, 'maxiter': MAX_ITER},
        tol=0.2)

    return res




#### Experiments

In [12]:
for n_layers in range(4, 13):
    print(f'RUNNING EXPERIMENT FOR {n_layers} LAYERS:')
    qc = gen_vqc_ansatz(n_qubits, n_layers)
    theta0 = [np.random.random() for _ in range(qc.num_parameters)]
    layer_res = run_vqc_exp(qc, n_qubits, n_layers, theta0, 0.2)

    no_mub_res[n_layers] = layer_res

RUNNING EXPERIMENT FOR 4 LAYERS:
RUNNING EXPERIMENT FOR 5 LAYERS:
RUNNING EXPERIMENT FOR 6 LAYERS:
RUNNING EXPERIMENT FOR 7 LAYERS:
RUNNING EXPERIMENT FOR 8 LAYERS:
RUNNING EXPERIMENT FOR 9 LAYERS:
RUNNING EXPERIMENT FOR 10 LAYERS:
RUNNING EXPERIMENT FOR 11 LAYERS:
RUNNING EXPERIMENT FOR 12 LAYERS:


#### Analysis

In [ ]:
no_mub_nfev ={k: v['nfev'] for k,v in no_mub_res.items()}

with open('no_mub_3.txt', 'w') as f:
    f.write(str(no_mub_res))
    f.write('\n\n=====NFEV (Number of Function Evals) Summary=====\n\n')
    f.write(str(no_mub_nfev))

## Mitigating Barren Plateaus using MUBs

### Loading the QASM MUB circuits into Qiskit

In [23]:
FROM_FILES = True
if FROM_FILES:
    paths = os.listdir(path_3_306)
    circuits = [qk.circuit.QuantumCircuit.from_qasm_file(os.path.join(path_3_306,path)) for path in paths if '.txt' in path]
else:
    circuits = [qk.circuit.QuantumCircuit.from_qasm_str(qasm_str) for qasm_str in qasm_3_306.values()]
    

### Experimenting with MUBs - prepending
This is an experiment in which a MUB transformation is applied *before* the ansatz circuits.

#### Experiment Functions

In [24]:
# This function gets a number i from 0 to (2^n)-1
# and returns a circuit that generates the state |i> when acting on |0>.
def get_comp_state_circ(state_idx: int, n_qubits: int) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits)
    bin_str = bin(state_idx)[2:].zfill(n_qubits)
    for i, ch in enumerate(bin_str):
        if ch == '1':
            qc.x(i)
    return qc

def run_vqc_exp_with_mub_prepend(ansatz_qc: QuantumCircuit, mub_qc: QuantumCircuit, n_qubits: int, n_layers: int, theta0: List[float], tol: float = 0.2) -> dict:
    mub_qc = mub_qc.copy()
    mub_qc.add_register(ClassicalRegister(n_qubits))
    mub_ansatz_qc = mub_qc.compose(ansatz_qc, qubits=range(n_qubits), inplace=False)
    assert mub_ansatz_qc != None

    res_dict = {}
    for i in range((2 ** n_qubits)):
        starting_qc = get_comp_state_circ(i, n_qubits)
        starting_qc.add_register(ClassicalRegister(n_qubits))
        full_qc = starting_qc.compose(mub_ansatz_qc, qubits=range(n_qubits), inplace=False)
        assert full_qc != None
        res_dict[i] = run_vqc_exp(full_qc, n_qubits, n_layers, theta0, tol)

    return res_dict

#### Experiments

In [ ]:
### Defining experiment values
n_qubits = 3
SHOTS = 8192
MAX_ITER = 10000
backend = Aer.get_backend('qasm_simulator')
qi = QuantumInstance(backend, shots=SHOTS)

print('=====Experimenting with prepended MUB states=====')
all_prepend_mub_results = {}
for n_layers in range(4,13):
    layer_results = {}
    print(f'---experimenting with {n_layers} layers---')
    ansatz = gen_vqc_ansatz(n_qubits, n_layers)
    theta0 = [np.random.random() for _ in range(ansatz.num_parameters)]
    for i, mub_circuit in enumerate(circuits):
        print(f'experimenting with MUB #{i+1}')
        res = run_vqc_exp_with_mub_prepend(ansatz, mub_circuit, n_qubits, n_layers, theta0, tol=0.2)
        # print(res)
        print(f'MUB #{i+1} done')
        layer_results[i] = res
    all_prepend_mub_results[n_layers] = layer_results

#### Analysis

In [ ]:
## Keeping NFEV results
prepend_mub_nfev = {}
for l, d_l in all_prepend_mub_results.items():
    print(f'nfev for {l} layers')
    min_nfev = min([min([state_res['nfev'] for state_res in d_mub.values()]) for d_mub in d_l.values()])
    print(f'minimal nfev for {l} layers is {min_nfev}')
    prepend_mub_nfev[l] = min_nfev

with open('prepend_mub_results_3.txt', 'w') as f:
    f.write(str(all_prepend_mub_results))
    f.write('\n\n=====NFEV (Number of Function Evals) Summary=====\n\n')
    f.write(str(prepend_mub_nfev))

### Experimenting with MUBs - appending
Now, instead of generating MUB states at the beginning, we add MUB transformations at the end.
Let's see how that goes.

Seeing as state initial sate preparation does not take place, there is no meaning for each individual state.

#### Experiment Functions

In [13]:
def run_vqc_exp_with_mub_append(ansatz_qc: QuantumCircuit, mub_qc: QuantumCircuit, n_qubits: int, n_layers: int, theta0: List[float], tol: float = 0.2) -> dict:
    mub_qc = mub_qc.copy()
    mub_qc.add_register(ClassicalRegister(n_qubits))
    mub_ansatz_qc = mub_qc.compose(ansatz_qc, qubits=range(n_qubits), inplace=False)
    assert mub_ansatz_qc != None
    
    return run_vqc_exp(mub_ansatz_qc, n_qubits, n_layers, theta0, tol)

#### Experiments

In [ ]:
### Defining experiment values
n_qubits = 3
SHOTS = 8192
MAX_ITER = 10000
backend = Aer.get_backend('qasm_simulator')
qi = QuantumInstance(backend, shots=SHOTS)

print('=====Experimenting with appended MUB states=====')
all_append_mub_results = {}
for n_layers in range(4,13):
    layer_results = {}
    print(f'---experimenting with {n_layers} layers---')
    ansatz = gen_vqc_ansatz(n_qubits, n_layers)
    theta0 = [np.random.random() for _ in range(ansatz.num_parameters)]
    for i, mub_circuit in enumerate(circuits):
        print(f'experimenting with MUB #{i+1}')
        res = run_vqc_exp_with_mub_append(ansatz, mub_circuit, n_qubits, n_layers, theta0, tol=0.2)
        # print(res)
        print(f'MUB #{i+1} done')
        layer_results[i] = res
    all_append_mub_results[n_layers] = layer_results

#### Analysis

In [ ]:
## Keeping NFEV results
append_mub_nfev = {}
for l, d_l in all_append_mub_results.items():
    print(f'nfev for {l} layers')
    min_nfev = min([mub_res['nfev'] for mub_res in d_l.values()])
    print(f'minimal nfev for {l} layers is {min_nfev}')
    append_mub_nfev[l] = min_nfev

with open('append_mub_results_3.txt', 'w') as f:
    f.write(str(all_append_mub_results))
    f.write('\n\n=====NFEV (Number of Function Evals) Summary=====\n\n')
    f.write(str(append_mub_nfev))

## Basic Barren Plateau Circuit for Transverse Ising VQE
To save myself some time, I'll just use the built-in VQE module.

In [25]:
def build_pauli_string(n_qubits: int, mat_locations: Dict[int, str]) -> str:
    assert n_qubits > 0
    H = ''
    for i in range(n_qubits):
        H += mat_locations.get(i, 'I')
    return H

def generate_transverse_ising_ham(n_qubits: int) -> PauliSumOp:
    # Build Individual X strings
    string_list = [(build_pauli_string(n_qubits, {i: 'X'}), random()) for i in range(n_qubits)]
    for i in range(n_qubits):
        string_list += [(build_pauli_string(n_qubits, {i: 'Z', j: 'Z'}), random()) for j in range(i+1, n_qubits)]
    return PauliSumOp.from_list(string_list, 1)

In [26]:
## For now, I'm using the standard qiskit hardware efficient ansatz (as Dekel suggested).
def gen_hardware_eff_ansatz(n_qubits: int, n_layers: int) -> QuantumCircuit:
    qc = QuantumCircuit(n_qubits)
    ansatz = EfficientSU2(n_qubits, reps=n_layers, entanglement='linear')
    qc.compose(ansatz, inplace=True)
    return qc

### Experimenting without MUBs

#### Experiment Functions

In [27]:
# Returns the number of function evaluations it took for the method to converge.
def run_vqe_exp(ansatz: QuantumCircuit, ham: PauliSumOp, theta0: Union[List[float], None] = None, tol: float = 0.2) -> VQEResult:
    optimizer = COBYLA(tol=tol)
    vqe = VQE(ansatz=ansatz, optimizer=optimizer, quantum_instance=qi, initial_point=np.asarray(theta0))
    result = vqe.compute_minimum_eigenvalue(operator=ham)
    return result

#### Experiments

In [9]:
### Defining experiment values
n_qubits = 3
SHOTS = 8192
MAX_ITER = 10000
backend = Aer.get_backend('qasm_simulator')
qi = QuantumInstance(backend, shots=SHOTS)

no_mub_res = {}

for n_layers in range(4, 13):
    print(f'RUNNING VQE EXPERIMENT FOR {n_layers} LAYERS:')
    # n_qubits = n_layers
    ham = generate_transverse_ising_ham(n_qubits)
    ansatz = gen_hardware_eff_ansatz(n_qubits, n_layers)
    theta0 = [np.random.random() for _ in range(ansatz.num_parameters)]
    layer_res = run_vqe_exp(ansatz, ham, theta0=theta0, tol=0.2)
    no_mub_res[n_layers] = layer_res

RUNNING VQE EXPERIMENT FOR 4 LAYERS:


/Users/ittayalfassi/opt/anaconda3/envs/qiskit/lib/python3.8/site-packages/sympy/core/expr.py:3949: SymPyDeprecationWarning: 

expr_free_symbols method has been deprecated since SymPy 1.9. See
https://github.com/sympy/sympy/issues/21494 for more info.

  SymPyDeprecationWarning(feature="expr_free_symbols method",


RUNNING VQE EXPERIMENT FOR 5 LAYERS:


/Users/ittayalfassi/opt/anaconda3/envs/qiskit/lib/python3.8/site-packages/sympy/core/expr.py:3949: SymPyDeprecationWarning: 

expr_free_symbols method has been deprecated since SymPy 1.9. See
https://github.com/sympy/sympy/issues/21494 for more info.

  SymPyDeprecationWarning(feature="expr_free_symbols method",


RUNNING VQE EXPERIMENT FOR 6 LAYERS:
RUNNING VQE EXPERIMENT FOR 7 LAYERS:
RUNNING VQE EXPERIMENT FOR 8 LAYERS:
RUNNING VQE EXPERIMENT FOR 9 LAYERS:
RUNNING VQE EXPERIMENT FOR 10 LAYERS:
RUNNING VQE EXPERIMENT FOR 11 LAYERS:
RUNNING VQE EXPERIMENT FOR 12 LAYERS:


#### Analysis

In [12]:
no_mub_nfev ={k: v.cost_function_evals for k,v in no_mub_res.items()}

with open('no_mub_3.txt', 'w', encoding='utf-8') as f:
    f.write(str({k: str(v) for k,v in no_mub_res.items()}))
    f.write('\n\n=====NFEV (Number of Function Evals) Summary=====\n\n')
    f.write(str(no_mub_nfev))

### Experimenting with MUBs - prepending (VQE)

#### Experiment Functions

In [28]:
def run_vqe_exp_with_mub_prepend(ansatz_qc: QuantumCircuit, mub_qc: QuantumCircuit, n_qubits: int, ham: PauliSumOp, theta0: Union[List[float], None] = None, tol: float = 0.2) -> dict:
    mub_qc = mub_qc.copy()
    mub_ansatz_qc = mub_qc.compose(ansatz_qc, qubits=range(n_qubits), inplace=False)
    assert mub_ansatz_qc != None

    res_dict = {}
    for i in range((2 ** n_qubits)):
        starting_qc = get_comp_state_circ(i, n_qubits)
        full_qc = starting_qc.compose(mub_ansatz_qc, qubits=range(n_qubits), inplace=False)
        assert full_qc != None
        res_dict[i] = run_vqe_exp(full_qc, ham, theta0=theta0, tol=tol)

    return res_dict

#### Experiments

In [30]:
### Defining experiment values
n_qubits = 3
SHOTS = 8192
MAX_ITER = 10000
backend = Aer.get_backend('qasm_simulator')
qi = QuantumInstance(backend, shots=SHOTS)
ham = generate_transverse_ising_ham(n_qubits)

print('=====Experimenting with prepended MUB states=====')
all_prepend_mub_results = {}
for n_layers in range(4,13):
    print(f'---experimenting with {n_layers} layers---')
    layer_results = {}
    ansatz = gen_hardware_eff_ansatz(n_qubits, n_layers)
    theta0 = [np.random.random() for _ in range(ansatz.num_parameters)]
    for i, mub_circuit in enumerate(circuits):
        print(f'experimenting with MUB #{i+1}')
        res = run_vqe_exp_with_mub_prepend(ansatz, mub_circuit, n_qubits, ham, theta0=theta0, tol=0.2)
        # print(res)
        print(f'MUB #{i+1} done')
        layer_results[i] = res
    all_prepend_mub_results[n_layers] = layer_results

=====Experimenting with prepended MUB states=====
---experimenting with 4 layers---
experimenting with MUB #1
MUB #1 done
experimenting with MUB #2
MUB #2 done
experimenting with MUB #3
MUB #3 done
experimenting with MUB #4
MUB #4 done
experimenting with MUB #5
MUB #5 done
experimenting with MUB #6
MUB #6 done
experimenting with MUB #7
MUB #7 done
experimenting with MUB #8
MUB #8 done
experimenting with MUB #9
MUB #9 done
---experimenting with 5 layers---
experimenting with MUB #1
MUB #1 done
experimenting with MUB #2
MUB #2 done
experimenting with MUB #3
MUB #3 done
experimenting with MUB #4
MUB #4 done
experimenting with MUB #5
MUB #5 done
experimenting with MUB #6
MUB #6 done
experimenting with MUB #7
MUB #7 done
experimenting with MUB #8
MUB #8 done
experimenting with MUB #9
MUB #9 done
---experimenting with 6 layers---
experimenting with MUB #1
MUB #1 done
experimenting with MUB #2
MUB #2 done
experimenting with MUB #3
MUB #3 done
experimenting with MUB #4
MUB #4 done
experimentin

#### Analysis

In [ ]:
## Keeping NFEV results
prepend_mub_nfev = {}
prepend_mub_adv = {}
for l, d_l in all_prepend_mub_results.items():
    print(f'nfev for {l} layers')
    min_nfev = min([min([state_res.cost_function_evals for state_res in d_mub.values()]) for d_mub in d_l.values()])
    layer_mub_adv = {}
    for mub_idx, d_mub in d_l.items():
        layer_mub_adv[mub_idx] = len([state_res.cost_function_evals for state_res in d_mub.values() if state_res.cost_function_evals > no_mub_nfev[l]])
    print(f'minimal nfev for {l} layers is {min_nfev}')
    prepend_mub_nfev[l] = min_nfev
    prepend_mub_adv[l] = layer_mub_adv

with open('prepend_mub_results_3.txt', 'w') as f:
    f.write(str(all_prepend_mub_results))
    f.write('\n\n=====NFEV (Number of Function Evals) Summary=====\n\n')
    f.write(str(prepend_mub_nfev))

### Experimenting with MUBs - appending

#### Experiment Functions

In [13]:
def run_vqe_exp_with_mub_append(ansatz_qc: QuantumCircuit, mub_qc: QuantumCircuit, n_qubits: int, ham: PauliSumOp, theta0: Union[List[float], None] = None, tol: float = 0.2) -> VQEResult:
    mub_qc = mub_qc.copy()
    mub_ansatz_qc = mub_qc.compose(ansatz_qc, qubits=range(n_qubits), inplace=False)
    assert mub_ansatz_qc != None
    
    return run_vqe_exp(mub_ansatz_qc, ham, theta0, tol)

#### Experiments

In [14]:
### Defining experiment values
n_qubits = 3
SHOTS = 8192
MAX_ITER = 10000
backend = Aer.get_backend('qasm_simulator')
qi = QuantumInstance(backend, shots=SHOTS)
ham = generate_transverse_ising_ham(n_qubits)

print('=====Experimenting with appended MUB states=====')
all_append_mub_results = {}
for n_layers in range(4,13):
    print(f'---experimenting with {n_layers} layers---')
    layer_results = {}
    ansatz = gen_hardware_eff_ansatz(n_qubits, n_layers)
    theta0 = [np.random.random() for _ in range(ansatz.num_parameters)]
    for i, mub_circuit in enumerate(circuits):
        print(f'experimenting with MUB #{i+1}')
        res = run_vqe_exp_with_mub_append(ansatz, mub_circuit, n_qubits, ham, theta0=theta0, tol=0.2)
        # print(res)
        print(f'MUB #{i+1} done')
        layer_results[i] = res
    all_append_mub_results[n_layers] = layer_results

=====Experimenting with appended MUB states=====
---experimenting with 4 layers---
experimenting with MUB #1
MUB #1 done
experimenting with MUB #2
MUB #2 done
experimenting with MUB #3
MUB #3 done
experimenting with MUB #4
MUB #4 done
experimenting with MUB #5
MUB #5 done
experimenting with MUB #6
MUB #6 done
experimenting with MUB #7
MUB #7 done
experimenting with MUB #8
MUB #8 done
experimenting with MUB #9
MUB #9 done
---experimenting with 5 layers---
experimenting with MUB #1
MUB #1 done
experimenting with MUB #2
MUB #2 done
experimenting with MUB #3
MUB #3 done
experimenting with MUB #4
MUB #4 done
experimenting with MUB #5
MUB #5 done
experimenting with MUB #6
MUB #6 done
experimenting with MUB #7
MUB #7 done
experimenting with MUB #8
MUB #8 done
experimenting with MUB #9
MUB #9 done
---experimenting with 6 layers---
experimenting with MUB #1
MUB #1 done
experimenting with MUB #2
MUB #2 done
experimenting with MUB #3
MUB #3 done
experimenting with MUB #4
MUB #4 done
experimenting

#### Analysis

In [15]:
def gen_statistics_appending(append_results_dict: Dict[int, Dict[any]],
no_mub_nfev: Dict[int, int],
filename: str = 'append_mub_results_3.txt') -> Dict[int, Dict[str, any]]:
    ## Keeping NFEV results
    append_mub_nfev = {}
    ## Keeping count of how many MUB transformations were advantageous
    append_mub_adv = {}
    for l, d_l in append_results_dict.items():
        print(f'nfev for {l} layers')
        min_nfev = min([mub_res.cost_function_evals for mub_res in d_l.values()])
        adv_states_count = len([mub_res.cost_function_evals for mub_res in d_l.values() if mub_res.cost_function_evals > no_mub_nfev[l]])
        print(f'minimal nfev for {l} layers is {min_nfev}')
        append_mub_nfev[l] = min_nfev
        append_mub_adv[l] = adv_states_count

    with open('append_mub_results_3.txt', 'w') as f:
        f.write(str(all_append_mub_results))
        f.write('\n\n=====NFEV (Number of Function Evals) Summary=====\n\n')
        f.write(str(append_mub_nfev))
        f.write('\n\n=====Number of Advantageous MUB trans. Summary===\n\n')
        f.write(str(append_mub_adv))


## Keeping NFEV results
append_mub_nfev = {}
## Keeping count of how many MUB transformations were advantageous
append_mub_adv = {}
for l, d_l in all_append_mub_results.items():
    print(f'nfev for {l} layers')
    min_nfev = min([mub_res.cost_function_evals for mub_res in d_l.values()])
    adv_states_count = len([mub_res.cost_function_evals for mub_res in d_l.values() if mub_res.cost_function_evals > no_mub_nfev[l]])
    print(f'minimal nfev for {l} layers is {min_nfev}')
    append_mub_nfev[l] = min_nfev
    append_mub_adv[l] = adv_states_count

with open('append_mub_results_3.txt', 'w') as f:
    f.write(str(all_append_mub_results))
    f.write('\n\n=====NFEV (Number of Function Evals) Summary=====\n\n')
    f.write(str(append_mub_nfev))
    f.write('\n\n=====Number of Advantageous MUB trans. Summary===\n\n')
    f.write(str(append_mub_adv))
    

nfev for 4 layers
minimal nfev for 4 layers is 116
nfev for 5 layers
minimal nfev for 5 layers is 136
nfev for 6 layers
minimal nfev for 6 layers is 140
nfev for 7 layers
minimal nfev for 7 layers is 180
nfev for 8 layers
minimal nfev for 8 layers is 212
nfev for 9 layers
minimal nfev for 9 layers is 250
nfev for 10 layers
minimal nfev for 10 layers is 238
nfev for 11 layers
minimal nfev for 11 layers is 262
nfev for 12 layers
minimal nfev for 12 layers is 290
